# 3D classification example based on DenseNet

This tutorial shows an example of 3D classification task based on DenseNet and array format transforms.

Here, the task is given to classify MR images into male/female.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Project-MONAI/tutorials/blob/main/3d_classification/torch/densenet_training_array.ipynb)

## Setup environment

In [1]:
#!python -c "import monai" || pip install -q "monai-weekly[nibabel, tqdm]"


## Setup imports

In [2]:
# Copyright 2020 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import logging
import os
import glob
import sys
import shutil
import tempfile
import datetime
import socket
import functools

from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import nibabel as nib

import torch
from torch.utils.tensorboard import SummaryWriter

from sklearn.metrics import f1_score, accuracy_score, balanced_accuracy_score, roc_auc_score

import monai
from monai.apps import download_and_extract
from monai.config import print_config
from monai.data import DataLoader, ImageDataset
from monai.transforms import (
    EnsureChannelFirst,
    Compose,
    RandRotate90,
    Resize,
    ScaleIntensity,
)

pin_memory = torch.cuda.is_available()
torch.backends.cudnn.benchmark = False #torch.cuda.is_available() # Set this to true if the code fails
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
print_config()

MONAI version: 1.0.1
Numpy version: 1.23.5
Pytorch version: 1.11.0
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 8271a193229fe4437026185e218d5b06f7c8ce69
MONAI __file__: /home/phcavelar/miniconda3/envs/shade2022/lib/python3.10/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 4.0.2
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
Pillow version: 9.2.0
Tensorboard version: 2.11.0
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.12.0
tqdm version: 4.64.1
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.4
pandas version: 1.5.2
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.mona

In [3]:
def get_predicted_values(model, loader):
    t_model_outputs = []
    t_test_labels = []
    for test_data in loader:
        test_images, test_labels = test_data[0].to(device), test_data[1].to(device)
        with torch.no_grad():
            val_outputs = model(test_images)
            t_model_outputs.append(val_outputs.cpu().detach().numpy())
            t_test_labels.append(test_labels.cpu().detach().numpy())
    conf_model_outputs = np.concatenate(t_model_outputs)
    conf_test_labels = np.concatenate(t_test_labels)
    return conf_model_outputs, conf_test_labels

def get_cm(conf_model_outputs, conf_test_labels, num_classes):
    conf_matrix = np.zeros((num_classes,num_classes))
    for i in range(num_classes):
        in_class_i = conf_test_labels==i
        for j in range(num_classes):
            in_class_i_predicted_in_class_j = sum(conf_model_outputs[in_class_i]==j)
            conf_matrix[i,j] = in_class_i_predicted_in_class_j
    return conf_matrix

In [ ]:
# Set data directory
#directory = os.environ.get("MONAI_DATA_DIRECTORY")
#root_dir = tempfile.mkdtemp() if directory is None else directory
root_dir = os.path.expanduser(os.path.expandvars("~/data/medicaldecathlon/"))
data_dir = os.path.join(root_dir, "Task10_Colon")
train_dataset_frailty_path = os.path.join(data_dir,"train_clean.csv")
test_dataset_frailty_path = os.path.join(data_dir,"test_clean.csv")
print(root_dir)

In [ ]:
train_images = sorted(
    glob.glob(os.path.join(data_dir, "imagesTr", "*.nii.gz")))
train_images[:5]

In [ ]:
test_image_paths = sorted(
    glob.glob(os.path.join(data_dir, "imagesTs", "*.nii.gz")))
test_image_paths[:5]

In [7]:
int_cols = ["Skeletal Muscle Wasting","Fat Excess","Bone density","Aortic Calcium","Liver fat","Pancreatic fat","Total Score"]

In [8]:
df_labels = pd.read_csv(train_dataset_frailty_path, index_col="PatientID").dropna().astype({col:int for col in int_cols})
df_labels.head()

,Skeletal Muscle Wasting,Fat Excess,Bone density,Aortic Calcium,Liver fat,Pancreatic fat,Total Score,Risk Category
PatientID,,,,,,,,
1,1,2,0,1,0,1,5,MEDIUM
5,0,0,0,0,1,0,1,LOW
6,0,1,0,0,0,0,1,LOW
7,1,2,0,0,0,1,4,MEDIUM
8,1,1,0,0,0,0,2,LOW


In [9]:
df_labels_test = pd.read_csv(test_dataset_frailty_path, index_col="PatientID").dropna().astype({col:int for col in int_cols})
df_labels_test.head()

,Skeletal Muscle Wasting,Fat Excess,Bone density,Aortic Calcium,Liver fat,Pancreatic fat,Total Score,Risk Category
PatientID,,,,,,,,
3,1,1,0,1,1,1,5,MEDIUM
4,1,1,0,1,1,1,5,MEDIUM
10,0,0,0,1,0,1,2,LOW
13,0,0,0,0,0,1,1,LOW
17,2,1,0,1,0,0,4,MEDIUM


In [10]:
df_labels.loc[df_labels["Risk Category"]=="LOW","Risk Category"] = 0
df_labels.loc[df_labels["Risk Category"]=="MEDIUM","Risk Category"] = 1
df_labels.loc[df_labels["Risk Category"]=="HIGH","Risk Category"] = 2

df_labels_test.loc[df_labels_test["Risk Category"]=="LOW","Risk Category"] = 0
df_labels_test.loc[df_labels_test["Risk Category"]=="MEDIUM","Risk Category"] = 1
df_labels_test.loc[df_labels_test["Risk Category"]=="HIGH","Risk Category"] = 2

In [11]:
def get_id_from_filepath(fpath):
    return int(os.path.basename(fpath).split("_")[1].split(".")[0])

In [12]:
df_labels.dtypes

Skeletal Muscle Wasting     int64
Fat Excess                  int64
Bone density                int64
Aortic Calcium              int64
Liver fat                   int64
Pancreatic fat              int64
Total Score                 int64
Risk Category              object
dtype: object

In [13]:
label_types = ["Skeletal Muscle Wasting", "Fat Excess", "Bone density", "Aortic Calcium", "Liver fat", "Pancreatic fat"]
label_type_weights = [2,2,1,1,1,1]

In [14]:
MIN_SLICES = 64

In [15]:
data_dicts = [
    {"image": image_name, "label": df_labels.loc[patient_id,label_types].astype(int)}
    for image_name,patient_id in zip(train_images,map(get_id_from_filepath,train_images))
    if patient_id in df_labels.index and nib.load(image_name).get_fdata().shape[2]>=MIN_SLICES
]

In [16]:
test_data_dicts = [
    {"image": image_name, "label": df_labels_test.loc[patient_id,label_types].astype(int)}
    for image_name,patient_id in zip(test_image_paths,map(get_id_from_filepath,test_image_paths))
    if patient_id in df_labels_test.index and nib.load(image_name).get_fdata().shape[2]>=MIN_SLICES
]
len(test_data_dicts)

55

In [40]:
# IXI dataset as a demo, downloadable from https://brain-development.org/ixi-dataset/
images = np.array([d["image"] for d in data_dicts])
# 2 binary labels for gender classification: man or woman
labels = np.array([d["label"] for d in data_dicts])

test_images = np.array([d["image"] for d in test_data_dicts])
# 2 binary labels for gender classification: man or woman
test_labels = np.array([d["label"] for d in test_data_dicts])


# Represent labels in one-hot format for binary classifier training,
# BCEWithLogitsLoss requires target to have same shape as input
#labels = torch.nn.functional.one_hot(torch.as_tensor(labels)).float()
num_labels = [int(max(labels[:,l]))+1 for l in range(labels.shape[1])] if isinstance(labels, np.ndarray) and len(labels.shape)>1 else int(max(labels))+1
num_labels_cumsum = np.cumsum(num_labels).tolist()
num_labels, num_labels_cumsum, labels[:5], test_labels[:5]

([4, 4, 2, 4, 2, 4],
 [4, 8, 10, 14, 16, 20],
 array([[0, 0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0, 0],
        [1, 2, 0, 0, 0, 1],
        [1, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 1]]),
 array([[1, 1, 0, 1, 1, 1],
        [1, 1, 0, 1, 1, 1],
        [0, 0, 0, 1, 0, 1],
        [0, 0, 0, 0, 0, 1],
        [2, 1, 0, 1, 0, 0]]))

In [18]:
shapes = [nib.load(datapoint["image"]).get_fdata().shape for datapoint in data_dicts]
np_shapes = np.stack(shapes)
minshapes = np.min(np_shapes, axis=0)
minshapes

array([512, 512,  68])

In [19]:
original_crop = 96
for newcrop in range(original_crop,2,-1):
    if newcrop <= min(minshapes):
        crop_shapes = tuple([newcrop]*3)
        break
crop_shapes

(68, 68, 68)

In [20]:
val_pct = 0.2
val_split = int(val_pct*len(labels))

# TODO: Stratified split
#possible_labels = sorted(list(set(labels)))
#proportion_in_labels = np.array([sum(labels==i)/len(labels) for i in possible_labels])
#val_per_labels = [int(l*val_split) for l in proportion_in_labels]
#proportion_in_labels, val_per_labels
val_split

21

In [21]:
label_weigths = None#1/proportion_in_labels[labels]

In [22]:
# TODO: Stratified split
#val_idx = np.concatenate([np.random.choice([i for i, l in enumerate(labels) if l==p], c, replace=False) for p,c in zip(possible_labels, val_per_labels)])
val_idx = np.random.choice([i for i in range(len(labels))], val_split, replace=False)
in_val = np.isin(np.arange(len(labels)),val_idx)
in_train = ~in_val
train_idx = np.arange(len(labels))[in_train]
val_idx

array([ 89,   1,  10,   5,  11,  96,  53, 102,  30, 106,  19,  77,  61,
        79,  49,  97,  31,  39,  63,  27,  82])

In [23]:
batch_size = 4#3
while (len(images)-val_split)%batch_size==1 or val_split%batch_size==1:
    batch_size +=1
    print("Changing batch size so that no batch has size 1")
batch_size

Changing batch size so that no batch has size 1
Changing batch size so that no batch has size 1


6

In [24]:
# Define transforms
train_transforms = Compose([ScaleIntensity(), EnsureChannelFirst(), Resize(crop_shapes), RandRotate90()])

val_transforms = Compose([ScaleIntensity(), EnsureChannelFirst(), Resize(crop_shapes)])

# Define nifti dataset, data loader
check_ds = ImageDataset(image_files=images, labels=labels, transform=train_transforms)
check_loader = DataLoader(check_ds, batch_size=3, num_workers=2, pin_memory=pin_memory)

im, label = monai.utils.misc.first(check_loader)
print(type(im), im.shape, label, label.shape)

# create a training data loader
train_ds = ImageDataset(image_files=images[train_idx].tolist(), labels=labels[train_idx], transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=pin_memory)

# create a validation data loader
val_ds = ImageDataset(image_files=images[val_idx].tolist(), labels=labels[val_idx], transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size=batch_size, num_workers=2, pin_memory=pin_memory)

<class 'monai.data.meta_tensor.MetaTensor'> (3, 1, 68, 68, 68) tensor([[0, 0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0, 0],
        [1, 2, 0, 0, 0, 1]]) torch.Size([3, 6])


In [25]:
len(images)

109

In [26]:
len(train_ds), len(val_ds)

(88, 21)

In [27]:
len(train_loader), len(val_loader)

(15, 4)

In [28]:
metrics_fns = {
    (s.func.__name__ if isinstance(s, functools.partial) else s.__name__).split("_score")[0]: s
    for s in [functools.partial(f1_score, average="micro"), accuracy_score, balanced_accuracy_score]
}
val_metric = "balanced_accuracy"
metrics_fns

{'f1': functools.partial(<function f1_score at 0x7ff0ea940940>, average='micro'),
 'accuracy': <function sklearn.metrics._classification.accuracy_score(y_true, y_pred, *, normalize=True, sample_weight=None)>,
 'balanced_accuracy': <function sklearn.metrics._classification.balanced_accuracy_score(y_true, y_pred, *, sample_weight=None, adjusted=False)>}

In [ ]:
# Create DenseNet121, CrossEntropyLoss and Adam optimizer
model = monai.networks.nets.DenseNet(spatial_dims=3, in_channels=1, out_channels=np.sum(num_labels)).to(device)

#loss_function = [torch.nn.CrossEntropyLoss(torch.tensor(1/proportion_in_labels, device=device, dtype=torch.float32)) for _ in label_types]
loss_function = [torch.nn.CrossEntropyLoss() for _ in label_types]
# loss_function = torch.nn.BCEWithLogitsLoss()  # also works with this data

optimizer = torch.optim.Adam(model.parameters(), 1e-4)

# start a typical PyTorch training
val_interval = 2
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = []
metric_values = []
writer = SummaryWriter(f"multitask_runs/{datetime.datetime.now():%Y-%m-%d_%H:%M:%S}_{socket.gethostname()}")
max_epochs = 256

num_labels_ranges = [0]+num_labels_cumsum

for epoch in range(max_epochs):
    try:
        print("-" * 10)
        print(f"epoch {epoch + 1}/{max_epochs}")
        model.train()
        epoch_loss = 0
        step = 0

        for batch_data in train_loader:
            step += 1
            b_inputs, b_labels = batch_data[0].to(device), batch_data[1].to(device)
            optimizer.zero_grad()
            outputs = model(b_inputs)
            loss = 0
            for l_idx,(loss_fn,loss_w,s,e) in enumerate(zip(loss_function, label_type_weights, num_labels_ranges[:-1],num_labels_ranges[1:])):
                loss += loss_fn(outputs[:,s:e], b_labels[:,l_idx])*loss_w
            loss /= sum(label_type_weights)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            epoch_len = len(train_ds) // train_loader.batch_size
            print(f"{step}/{epoch_len}, train_loss: {loss.item():.4f}")
            writer.add_scalar("train_loss", loss.item(), epoch_len * epoch + step)

        epoch_loss /= step
        epoch_loss_values.append(epoch_loss)
        print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

        if (epoch + 1) % val_interval == 0:
            model.eval()

            num_correct = 0.0
            metric_count = 0
            v_pred_raw, v_label = get_predicted_values(model, val_loader)
            v_pred = np.stack([
                    v_pred_raw[:,s:e].argmax(axis=1)
                    for (s,e) in zip(num_labels_ranges[:-1],num_labels_ranges[1:])
                ],
                axis=1,
            )

            all_metrics_per_label = {
                metric: [] for metric in metrics_fns
            }
            
            for l_idx, l in enumerate(label_types):
                for metric in all_metrics_per_label:
                    metric_value = metrics_fns[metric](v_label[:,l_idx], v_pred[:,l_idx])
                    writer.add_scalar(f"{l} {metric}",metric_value, epoch + 1)
                    all_metrics_per_label[metric].append(metric_value)
                cm_counts = get_cm(v_pred[:,l_idx], v_label[:,l_idx], num_labels[l_idx])
                cm_pct = cm_counts/cm_counts.sum(axis=1,keepdims=True)
                print(cm_pct)
                for i in range(num_labels[l_idx]):
                    for j in range(num_labels[l_idx]):
                        writer.add_scalar(f"{l} count l{i}_p{j}",cm_counts[i,j], epoch + 1)
                        writer.add_scalar(f"{l} pct l{i}_p{j}",cm_pct[i,j], epoch + 1)

            metric = np.mean(all_metrics_per_label[val_metric])
            metric_values.append(metric)

            if metric > best_metric:
                best_metric = metric
                best_metric_epoch = epoch + 1
                torch.save(model.state_dict(), "multitask_best_metric_model_classification3d_array.pth")
                print("saved new best metric model")

            print(f"Current epoch: {epoch+1} current {val_metric}: {metric:.4f} ")
            print(f"Best {val_metric}: {best_metric:.4f} at epoch {best_metric_epoch}")
            for metric in all_metrics_per_label:
                writer.add_scalar(f"val_{metric}", np.mean(all_metrics_per_label[metric]), epoch + 1)
    except KeyboardInterrupt:
        break

print(f"Training completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")
torch.save(model.state_dict(), "multitask_last_model_classification3d_array.pth")
writer.close()

# Occlusion sensitivity
One method for trying to visualise why the network made a given prediction is occlusion sensitivity. We occlude part of the image, and see how the probability of a given prediction changes. We then iterate over the image, moving the occluded portion as we go, and in doing so we build up a sensitivity map detailing which areas were the most important in making the decision.

#### Bounds
If we were to test the occlusion centred on all voxels in our image, we would have to do `torch.prod(im.shape) = 96^3 = ~1e6` predictions. We can use the bounding box to only to the estimations in a region of interest, for example over one slice.

To do this, we simply give the bounding box as `(minC,maxC,minD,maxD,minH,maxH,minW,maxW)`. We can use `-1` for any value to use its full extent (`0` and `im.shape-1` for min's and max's, respectively).

#### Output
The output image in this example will look fairly bad, since our network hasn't been trained for very long. Training for longer should improve the quality of the occlusion map.

In [41]:
# create a validation data loader
test_ds = ImageDataset(image_files=test_images, labels=test_labels, transform=val_transforms)
test_loader = DataLoader(test_ds, batch_size=2, num_workers=2, pin_memory=pin_memory)

train_ds = ImageDataset(image_files=images[train_idx].tolist(), labels=labels[train_idx], transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=pin_memory)

In [42]:
len(test_images)

55

In [36]:
v_pred_raw, v_label = get_predicted_values(model, train_loader)
v_pred = np.stack([
        v_pred_raw[:,s:e].argmax(axis=1)
        for (s,e) in zip(num_labels_ranges[:-1],num_labels_ranges[1:])
    ],
    axis=1,
)

all_metrics_per_label = {
    metric: [] for metric in metrics_fns
}

for l_idx, l in enumerate(label_types):
    for metric in all_metrics_per_label:
        metric_value = metrics_fns[metric](v_label[:,l_idx], v_pred[:,l_idx])
        print(f"train {l} {metric}",metric_value)
        all_metrics_per_label[metric].append(metric_value)
    cm_counts = get_cm(v_pred[:,l_idx], v_label[:,l_idx], num_labels[l_idx])
    cm_pct = cm_counts/cm_counts.sum(axis=1,keepdims=True)
    print(cm_counts)
    print(cm_pct)

for metric in all_metrics_per_label:
    print(f"train_{metric}", np.mean(all_metrics_per_label[metric]))

train Skeletal Muscle Wasting f1 1.0
train Skeletal Muscle Wasting accuracy 1.0
train Skeletal Muscle Wasting balanced_accuracy 1.0
[[31.  0.  0.  0.]
 [ 0. 37.  0.  0.]
 [ 0.  0. 19.  0.]
 [ 0.  0.  0.  1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
train Fat Excess f1 1.0
train Fat Excess accuracy 1.0
train Fat Excess balanced_accuracy 1.0
[[33.  0.  0.  0.]
 [ 0. 22.  0.  0.]
 [ 0.  0. 32.  0.]
 [ 0.  0.  0.  1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
train Bone density f1 1.0
train Bone density accuracy 1.0
train Bone density balanced_accuracy 1.0
[[78.  0.]
 [ 0. 10.]]
[[1. 0.]
 [0. 1.]]
train Aortic Calcium f1 1.0
train Aortic Calcium accuracy 1.0
train Aortic Calcium balanced_accuracy 1.0
[[42.  0.  0.  0.]
 [ 0. 31.  0.  0.]
 [ 0.  0. 10.  0.]
 [ 0.  0.  0.  5.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
train Liver fat f1 0.9886363636363636
train Liver fat accuracy 0.9886363636363636
train Liver fat balanced_accuracy

In [37]:
v_pred_raw, v_label = get_predicted_values(model, val_loader)
v_pred = np.stack([
        v_pred_raw[:,s:e].argmax(axis=1)
        for (s,e) in zip(num_labels_ranges[:-1],num_labels_ranges[1:])
    ],
    axis=1,
)

all_metrics_per_label = {
    metric: [] for metric in metrics_fns
}

for l_idx, l in enumerate(label_types):
    for metric in all_metrics_per_label:
        metric_value = metrics_fns[metric](v_label[:,l_idx], v_pred[:,l_idx])
        print(f"val {l} {metric}",metric_value)
        all_metrics_per_label[metric].append(metric_value)
    cm_counts = get_cm(v_pred[:,l_idx], v_label[:,l_idx], num_labels[l_idx])
    cm_pct = cm_counts/cm_counts.sum(axis=1,keepdims=True)
    print(cm_counts)
    print(cm_pct)

for metric in all_metrics_per_label:
    print(f"val_{metric}", np.mean(all_metrics_per_label[metric]))

val Skeletal Muscle Wasting f1 0.47619047619047616
val Skeletal Muscle Wasting accuracy 0.47619047619047616
val Skeletal Muscle Wasting balanced_accuracy 0.48412698412698413
[[2. 5. 0. 0.]
 [0. 4. 2. 0.]
 [0. 4. 4. 0.]
 [0. 0. 0. 0.]]
[[0.28571429 0.71428571 0.         0.        ]
 [0.         0.66666667 0.33333333 0.        ]
 [0.         0.5        0.5        0.        ]
 [       nan        nan        nan        nan]]
val Fat Excess f1 0.6190476190476191
val Fat Excess accuracy 0.6190476190476191
val Fat Excess balanced_accuracy 0.5904761904761905
[[4. 3. 0. 0.]
 [1. 2. 1. 0.]
 [1. 2. 7. 0.]
 [0. 0. 0. 0.]]
[[0.57142857 0.42857143 0.         0.        ]
 [0.25       0.5        0.25       0.        ]
 [0.1        0.2        0.7        0.        ]
 [       nan        nan        nan        nan]]
val Bone density f1 0.8571428571428571
val Bone density accuracy 0.8571428571428571
val Bone density balanced_accuracy 0.5
[[18.  0.]
 [ 3.  0.]]
[[1. 0.]
 [1. 0.]]
val Aortic Calcium f1 0.42857

/tmp/ipykernel_9284/1428580693.py:19: RuntimeWarning: invalid value encountered in divide
  cm_pct = cm_counts/cm_counts.sum(axis=1,keepdims=True)
/tmp/ipykernel_9284/1428580693.py:19: RuntimeWarning: invalid value encountered in divide
  cm_pct = cm_counts/cm_counts.sum(axis=1,keepdims=True)


In [43]:
v_pred_raw, v_label = get_predicted_values(model, test_loader)
v_pred = np.stack([
        v_pred_raw[:,s:e].argmax(axis=1)
        for (s,e) in zip(num_labels_ranges[:-1],num_labels_ranges[1:])
    ],
    axis=1,
)

all_metrics_per_label = {
    metric: [] for metric in metrics_fns
}

for l_idx, l in enumerate(label_types):
    for metric in all_metrics_per_label:
        metric_value = metrics_fns[metric](v_label[:,l_idx], v_pred[:,l_idx])
        print(f"test {l} {metric}",metric_value)
        all_metrics_per_label[metric].append(metric_value)
    cm_counts = get_cm(v_pred[:,l_idx], v_label[:,l_idx], num_labels[l_idx])
    cm_pct = cm_counts/cm_counts.sum(axis=1,keepdims=True)
    print(cm_counts)
    print(cm_pct)

for metric in all_metrics_per_label:
    print(f"test_{metric}", np.mean(all_metrics_per_label[metric]))

test Skeletal Muscle Wasting f1 0.34545454545454546
test Skeletal Muscle Wasting accuracy 0.34545454545454546
test Skeletal Muscle Wasting balanced_accuracy 0.26699604743083005
[[ 7. 12.  4.  0.]
 [ 6.  8.  5.  1.]
 [ 1.  6.  4.  0.]
 [ 0.  0.  1.  0.]]
[[0.30434783 0.52173913 0.17391304 0.        ]
 [0.3        0.4        0.25       0.05      ]
 [0.09090909 0.54545455 0.36363636 0.        ]
 [0.         0.         1.         0.        ]]
test Fat Excess f1 0.5454545454545454
test Fat Excess accuracy 0.5454545454545454
test Fat Excess balanced_accuracy 0.5524720893141946
[[15.  5.  5.  0.]
 [ 3.  8.  8.  0.]
 [ 1.  3.  7.  0.]
 [ 0.  0.  0.  0.]]
[[0.6        0.2        0.2        0.        ]
 [0.15789474 0.42105263 0.42105263 0.        ]
 [0.09090909 0.27272727 0.63636364 0.        ]
 [       nan        nan        nan        nan]]
test Bone density f1 0.8545454545454545
test Bone density accuracy 0.8545454545454545
test Bone density balanced_accuracy 0.47959183673469385
[[47.  2.]
 [ 

/tmp/ipykernel_9284/1817018252.py:19: RuntimeWarning: invalid value encountered in divide
  cm_pct = cm_counts/cm_counts.sum(axis=1,keepdims=True)


In [44]:
model.load_state_dict(torch.load("multitask_best_metric_model_classification3d_array.pth"))

<All keys matched successfully>

In [45]:
v_pred_raw, v_label = get_predicted_values(model, train_loader)
v_pred = np.stack([
        v_pred_raw[:,s:e].argmax(axis=1)
        for (s,e) in zip(num_labels_ranges[:-1],num_labels_ranges[1:])
    ],
    axis=1,
)

all_metrics_per_label = {
    metric: [] for metric in metrics_fns
}

for l_idx, l in enumerate(label_types):
    for metric in all_metrics_per_label:
        metric_value = metrics_fns[metric](v_label[:,l_idx], v_pred[:,l_idx])
        print(f"train {l} {metric}",metric_value)
        all_metrics_per_label[metric].append(metric_value)
    cm_counts = get_cm(v_pred[:,l_idx], v_label[:,l_idx], num_labels[l_idx])
    cm_pct = cm_counts/cm_counts.sum(axis=1,keepdims=True)
    print(cm_counts)
    print(cm_pct)

for metric in all_metrics_per_label:
    print(f"train_{metric}", np.mean(all_metrics_per_label[metric]))

train Skeletal Muscle Wasting f1 1.0
train Skeletal Muscle Wasting accuracy 1.0
train Skeletal Muscle Wasting balanced_accuracy 1.0
[[31.  0.  0.  0.]
 [ 0. 37.  0.  0.]
 [ 0.  0. 19.  0.]
 [ 0.  0.  0.  1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
train Fat Excess f1 1.0
train Fat Excess accuracy 1.0
train Fat Excess balanced_accuracy 1.0
[[33.  0.  0.  0.]
 [ 0. 22.  0.  0.]
 [ 0.  0. 32.  0.]
 [ 0.  0.  0.  1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
train Bone density f1 1.0
train Bone density accuracy 1.0
train Bone density balanced_accuracy 1.0
[[78.  0.]
 [ 0. 10.]]
[[1. 0.]
 [0. 1.]]
train Aortic Calcium f1 1.0
train Aortic Calcium accuracy 1.0
train Aortic Calcium balanced_accuracy 1.0
[[42.  0.  0.  0.]
 [ 0. 31.  0.  0.]
 [ 0.  0. 10.  0.]
 [ 0.  0.  0.  5.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
train Liver fat f1 1.0
train Liver fat accuracy 1.0
train Liver fat balanced_accuracy 1.0
[[67.  0.]
 [ 0. 21.]]
[[

In [46]:
v_pred_raw, v_label = get_predicted_values(model, val_loader)
v_pred = np.stack([
        v_pred_raw[:,s:e].argmax(axis=1)
        for (s,e) in zip(num_labels_ranges[:-1],num_labels_ranges[1:])
    ],
    axis=1,
)

all_metrics_per_label = {
    metric: [] for metric in metrics_fns
}

for l_idx, l in enumerate(label_types):
    for metric in all_metrics_per_label:
        metric_value = metrics_fns[metric](v_label[:,l_idx], v_pred[:,l_idx])
        print(f"val {l} {metric}",metric_value)
        all_metrics_per_label[metric].append(metric_value)
    cm_counts = get_cm(v_pred[:,l_idx], v_label[:,l_idx], num_labels[l_idx])
    cm_pct = cm_counts/cm_counts.sum(axis=1,keepdims=True)
    print(cm_counts)
    print(cm_pct)

for metric in all_metrics_per_label:
    print(f"val_{metric}", np.mean(all_metrics_per_label[metric]))

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff193ec2050>
Traceback (most recent call last):
  File "/home/phcavelar/miniconda3/envs/shade2022/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/home/phcavelar/miniconda3/envs/shade2022/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1341, in _shutdown_workers
    if w.is_alive():
  File "/home/phcavelar/miniconda3/envs/shade2022/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7ff193ec2050>
    Traceback (most recent call last):
assert self._parent_pid == os.getpid(), 'can only test a child process'  File "/home/phcavelar/miniconda3/envs/shade2022/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1358, in __del__

    AssertionErrorself._shutdown_workers(): 
can only test a child process  File "/home/phcavela

val Skeletal Muscle Wasting f1 0.42857142857142855
val Skeletal Muscle Wasting accuracy 0.42857142857142855
val Skeletal Muscle Wasting balanced_accuracy 0.4365079365079365
[[1. 5. 1. 0.]
 [0. 4. 2. 0.]
 [0. 4. 4. 0.]
 [0. 0. 0. 0.]]
[[0.14285714 0.71428571 0.14285714 0.        ]
 [0.         0.66666667 0.33333333 0.        ]
 [0.         0.5        0.5        0.        ]
 [       nan        nan        nan        nan]]
val Fat Excess f1 0.7619047619047619
val Fat Excess accuracy 0.7619047619047619
val Fat Excess balanced_accuracy 0.769047619047619
[[6. 1. 0. 0.]
 [1. 3. 0. 0.]
 [0. 3. 7. 0.]
 [0. 0. 0. 0.]]
[[0.85714286 0.14285714 0.         0.        ]
 [0.25       0.75       0.         0.        ]
 [0.         0.3        0.7        0.        ]
 [       nan        nan        nan        nan]]
val Bone density f1 0.8571428571428571
val Bone density accuracy 0.8571428571428571
val Bone density balanced_accuracy 0.5
[[18.  0.]
 [ 3.  0.]]
[[1. 0.]
 [1. 0.]]
val Aortic Calcium f1 0.5714285

/tmp/ipykernel_9284/1428580693.py:19: RuntimeWarning: invalid value encountered in divide
  cm_pct = cm_counts/cm_counts.sum(axis=1,keepdims=True)
/tmp/ipykernel_9284/1428580693.py:19: RuntimeWarning: invalid value encountered in divide
  cm_pct = cm_counts/cm_counts.sum(axis=1,keepdims=True)


In [47]:
v_pred_raw, v_label = get_predicted_values(model, test_loader)
v_pred = np.stack([
        v_pred_raw[:,s:e].argmax(axis=1)
        for (s,e) in zip(num_labels_ranges[:-1],num_labels_ranges[1:])
    ],
    axis=1,
)

all_metrics_per_label = {
    metric: [] for metric in metrics_fns
}

for l_idx, l in enumerate(label_types):
    for metric in all_metrics_per_label:
        metric_value = metrics_fns[metric](v_label[:,l_idx], v_pred[:,l_idx])
        print(f"test {l} {metric}",metric_value)
        all_metrics_per_label[metric].append(metric_value)
    cm_counts = get_cm(v_pred[:,l_idx], v_label[:,l_idx], num_labels[l_idx])
    cm_pct = cm_counts/cm_counts.sum(axis=1,keepdims=True)
    print(cm_counts)
    print(cm_pct)

for metric in all_metrics_per_label:
    print(f"test_{metric}", np.mean(all_metrics_per_label[metric]))

test Skeletal Muscle Wasting f1 0.34545454545454546
test Skeletal Muscle Wasting accuracy 0.34545454545454546
test Skeletal Muscle Wasting balanced_accuracy 0.27188735177865614
[[ 4. 15.  4.  0.]
 [ 3. 11.  6.  0.]
 [ 1.  6.  4.  0.]
 [ 0.  0.  1.  0.]]
[[0.17391304 0.65217391 0.17391304 0.        ]
 [0.15       0.55       0.3        0.        ]
 [0.09090909 0.54545455 0.36363636 0.        ]
 [0.         0.         1.         0.        ]]
test Fat Excess f1 0.509090909090909
test Fat Excess accuracy 0.509090909090909
test Fat Excess balanced_accuracy 0.525933014354067
[[16.  5.  4.  0.]
 [ 3.  4. 12.  0.]
 [ 2.  1.  8.  0.]
 [ 0.  0.  0.  0.]]
[[0.64       0.2        0.16       0.        ]
 [0.15789474 0.21052632 0.63157895 0.        ]
 [0.18181818 0.09090909 0.72727273 0.        ]
 [       nan        nan        nan        nan]]
test Bone density f1 0.7818181818181819
test Bone density accuracy 0.7818181818181819
test Bone density balanced_accuracy 0.5119047619047619
[[42.  7.]
 [ 5.  

/tmp/ipykernel_9284/1817018252.py:19: RuntimeWarning: invalid value encountered in divide
  cm_pct = cm_counts/cm_counts.sum(axis=1,keepdims=True)


## Cleanup data directory

Remove directory if a temporary was used.